In [6]:
##Load metadata from file

import ast

d = {}
nary = {}
with open("metadata_trial4.txt", 'r') as f:
    for line in f:
        d = (ast.literal_eval(line))
        nary.update({d['id'] : d})

In [3]:
#Filter terms that have children and aspect

filnary = {}
kilnary = {}
nochild = {}

for key, value in nary.items():
    if value.get('children') != None:
        filnary = ({'id' : value['id'], 'children' : value['children']})
        kilnary.update({key : filnary})
    else:
        filnary = ({'id' : value['id'],})
        nochild.update({key : filnary})

In [8]:
#get parents

for key, value in kilnary.items():
    for item in value['children']:
        if nary[item['id']].get('parent') == None:
            nary[item['id']]['parent'] = []
            nary[item['id']]['parent'].append({'id' : key, 'relation' : item['relation']})
        else:
            nary[item['id']]['parent'].append({'id' : key, 'relation' : item['relation']})
            
for key, value in nary.items():
    if value.get('parent') == None:
        value['parent'] = []

In [10]:
#empty children class for nochilds

for key, value in nary.items():
    if value.get('children') == None:
        value['children'] = []

In [11]:
for k,v in nary.items():
    for item in v['parent']:
        for item2 in v['children']:
            if item == item2:
                print(item)

{'id': 'GO:0016047', 'relation': 'is_a'}
{'id': 'GO:0009603', 'relation': 'is_a'}


In [5]:
#get rid of the two motherfuckers

for k,v in nary.items():
    for item in v['parent']:
        for item2 in v['children']:
            if item == item2:
                del(v['parent'][v['parent'].index(item)])
                del(v['children'][v['children'].index(item)])

In [27]:
#remove anything that is not a biological_process or molecular_function

c = 0
keys = []
for key, value in nary.items():
    if value['aspect'] != 'biological_process' and value['aspect'] != 'molecular_function':
        keys.append(key)
        c+=1
print(c)
for i in keys:
    del nary[i]

4367


In [6]:
#import owlready2

import types
from owlready2 import *
onto = get_ontology("http://test.org/onto.owl")

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


In [7]:
#define core classes

with onto:
    class therapeutic_agent(Thing):
        namespace = onto
    class biological_process_main(Thing):
        namespace = onto
    class molecular_function_main(Thing):
        namespace = onto
    class drug(Thing):
        namespace = onto

In [8]:
def recursive_classmaker_down(term):
    for item in nary[term]['children']:
        if item['relation'] == 'is_a':
            with onto:
                nc = types.new_class(item['id'], (getattr(onto, term),))
            recursive_classmaker_down(item['id'])
    return 0

In [9]:
#Using classmaker to make FTC classes

for key, value in nary.items():
    if value['parent'] == [] and value['aspect'] == 'biological_process':
        with onto:
            nc = types.new_class(key, (biological_process_main,))
        for item in value['children']:
            if item['relation'] == 'is_a':
                with onto:
                    nc = types.new_class(item['id'], (getattr(onto, key),))
                recursive_classmaker_down(item['id'])
    if value['parent'] == [] and value['aspect'] == 'molecular_function':
        with onto:
            nc = types.new_class(key, (molecular_function_main,))
        for item in value['children']:
            if item['relation'] == 'is_a':                
                with onto:
                    nc = types.new_class(item['id'], (getattr(onto, key),))
                recursive_classmaker_down(item['id'])
        


In [61]:
#list of classes created

GO_classes_created = [str(x)[5:] for x in list(onto.classes())]
len(GO_classes_created)

42986

In [62]:
for x in GO_classes_created:
    if x[2] != ':':
        print(x)

therapeutic_agent
biological_process_main
molecular_function_main
drug
fuck


In [63]:
#test number of expected classes to be created by the algorithm above

c = 0

for k, v in nary.items():
    if v['parent'] == []:
        c+=1
    elif 'is_a' in [a['relation'] for a in v['parent']]:
        c+=1
        
print(c)

classes_not_created = []

for k, v in nary.items():
    if k not in GO_classes_created:
        classes_not_created.append(k)


42975


In [64]:
classes_not_created

[]

In [55]:
for x in classes_not_created:
    for a in nary[x]['parent']:
        if a['id'] in GO_classes_created:
            with onto:
                nc = types.new_class(x, (getattr(onto, a['id']),))
#     if x in [y['relation'] for y in nary[x]['parent']]
#     if nary[x]['parent'] == [] and value['aspect'] == 'biological_process':

In [10]:
with onto:
    nc = types.new_class('fuck', (onto.therapeutic_agent,))

In [11]:
onto.fuck

onto.fuck

In [1]:
l = len(list(onto.classes()))

NameError: name 'onto' is not defined

In [13]:
for i in l:
    if str(i)[5:7] == "GO":
        for item in nary[str(i)[5:]]['children']:
            for ancestor in list(i.ancestors()):
                if item['id'] == str(ancestor)[5:]:
                    print(item['id'])

NameError: name 'l' is not defined

In [ ]:
#conflict checker

for k,v in nary.items():
    for item in v['parent']:
        for item2 in v['children']:
            if item == item2:
                print(item)

In [ ]:
#conflict checker

for k,v in nary.items():
    for item in v['parent']:
        for item2 in v['children']:
            if item == item2:
                del(v['parent'][v['parent'].index(item)])
                del(v['children'][v['children'].index(item)])

In [68]:
list(onto.classes())

[onto.therapeutic_agent,
 onto.biological_process_main,
 onto.molecular_function_main,
 onto.drug,
 onto.GO:0003674,
 onto.GO:0045182,
 onto.GO:0090079,
 onto.GO:0000900,
 onto.GO:0008494,
 onto.GO:0008135,
 onto.GO:0003743,
 onto.GO:0008079,
 onto.GO:0003747,
 onto.GO:0016150,
 onto.GO:0016149,
 onto.GO:0003746,
 onto.GO:0004694,
 onto.GO:0030371,
 onto.GO:0000901,
 onto.GO:0045183,
 onto.GO:0140110,
 onto.GO:0001072,
 onto.GO:0003711,
 onto.GO:0003700,
 onto.GO:0001217,
 onto.GO:0001227,
 onto.GO:0001216,
 onto.GO:0001228,
 onto.GO:0098531,
 onto.GO:0004879,
 onto.GO:0000981,
 onto.GO:0034246,
 onto.GO:0043856,
 onto.GO:0001073,
 onto.GO:0140223,
 onto.GO:0001181,
 onto.GO:0000995,
 onto.GO:0016251,
 onto.GO:0001070,
 onto.GO:0001134,
 onto.GO:0001186,
 onto.GO:0001004,
 onto.GO:0001135,
 onto.GO:0016987,
 onto.GO:0003712,
 onto.GO:0003713,
 onto.GO:0030374,
 onto.GO:0030375,
 onto.GO:0003714,
 onto.GO:0016989,
 onto.GO:0140313,
 onto.GO:0140315,
 onto.GO:0140319,
 onto.GO:0140320,
 

In [ ]:
l[5].ancestors()

In [ ]:
list(onto.biological_process.subclasses())

In [ ]:
abc = onto.behavior("test_abc")

In [12]:
nary['GO:0000139']

{'id': 'GO:0000139',
 'isObsolete': False,
 'name': 'Golgi membrane',
 'definition': {'text': 'The lipid bilayer surrounding any of the compartments of the Golgi apparatus.'},
 'children': [{'id': 'GO:0031228', 'relation': 'part_of'},
  {'id': 'GO:0090498', 'relation': 'part_of'},
  {'id': 'GO:0098548', 'relation': 'part_of'},
  {'id': 'GO:0048194', 'relation': 'occurs_in'},
  {'id': 'GO:0098547', 'relation': 'part_of'},
  {'id': 'GO:0032588', 'relation': 'is_a'},
  {'id': 'GO:0030173', 'relation': 'part_of'},
  {'id': 'GO:0090114', 'relation': 'occurs_in'},
  {'id': 'GO:0035964', 'relation': 'occurs_in'},
  {'id': 'GO:0032580', 'relation': 'is_a'}],
 'aspect': 'cellular_component',
 'usage': 'Unrestricted',
 'parent': [{'id': 'GO:0005794', 'relation': 'part_of'},
  {'id': 'GO:0098588', 'relation': 'is_a'},
  {'id': 'GO:0098791', 'relation': 'part_of'}]}

In [67]:
onto.save(file = "t3test2", format = "rdfxml")